In [1]:
import os
from dotenv import load_dotenv

from src.agents.graph_qa import GraphAgentResponder
from src.config import LLMConf, EmbedderConf, KnowledgeGraphConfig
from src.graph.knowledge_graph import KnowledgeGraph
from src.ingestion.embedder import ChunkEmbedder


env = load_dotenv('config.env')

In [2]:
kg_config = KnowledgeGraphConfig(
    uri=os.getenv("NEO4J_URI"),
    user=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name='vector'
)

llm_conf = LLMConf(
    type="ollama",
    model="llama3.2:latest", 
    temperature=0.0, 
)

embedder_conf = EmbedderConf(
    type="ollama",
    model="mxbai-embed-large",
)

In [3]:
embedder = ChunkEmbedder(conf=embedder_conf)

knowledge_graph = KnowledgeGraph(
    conf=kg_config, 
    embeddings_model=embedder.embeddings
)

knowledge_graph._driver.verify_connectivity()

knowledge_graph._driver.verify_authentication()

True

In [4]:
responder = GraphAgentResponder(
    qa_llm_conf=llm_conf, # TODO try different LLMs
    cypher_llm_conf=llm_conf,
    graph=knowledge_graph
)

In [5]:
query = "Which Countries have received RRF payments?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


'The countries that have received RRF payments are:\n\n1. Czechia\n2. Germany\n3. Italy\n4. Portugal\n5. Romania'

In [6]:
query = "What countries are mentioned in the graph?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Country) RETURN c.name
Full Context:
[{'c.name': 'Germany'}, {'c.name': 'Italy'}, {'c.name': 'Portugal'}, {'c.name': 'Romania'}, {'c.name': 'Czechia'}]

> Finished chain.


'The countries mentioned in the graph are:\n\n1. Germany\n2. Italy\n3. Portugal\n4. Romania\n5. Czechia'

In [7]:
query = "When was the request for payment from Germany submitted?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:MENTIONS]->(:Country {name: "Germany"})-[:HAS_RECEIVED_€11.39_BILLION]->(g:Country)
OPTIONAL MATCH (g)-[:PAYMENT]->()
RETURN g.recovery_plan_allocation


'The request for payment from Germany was submitted on 13 September 2024.'

In [8]:
query = "What happened in the baltic sea?"
responder.answer(query)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:MENTIONS]->(:Country {name: "Baltic Sea"})-[]


"There has been a series of suspected attacks on critical infrastructure in the Baltic Sea, including damaged electricity and data cables, which is believed to be part of Russia's shadow fleet threatening security and the environment. The European Commission is working with Finnish authorities to investigate and strengthen efforts to protect undersea cables, including enhanced information exchange, new detection technologies, and international cooperation."